Generating input data for HW3


Using data files from HW1 and HW2, generate inputs for d3 visualizations  
inputs:  
featureTable_withCluster.csv from HW1,  
new_attacks.tsv from HW2  

outputs:  
ZoomableTreeMapJson_SunValues.json,  
PieChartInput.csv,  
misspellingInput.csv,  
randomCapitalInput.csv,  
Heatmap_CrePhi.tsv,  
Heatmap_Malware.tsv,  
Heatmap_Reconn.tsv,  
Heatmap_SocEng.tsv,  
wordtable.csv


# import packages

In [1]:
import csv
import json
import numpy as np
import pandas as pd
import copy
from datetime import datetime
import calendar
import nltk
nltk.download('all')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

# D3 visualization
Zoomable Tree map

In our Zoomable tree map, the region is jaccardCluster and the subregion is the attack type.
key value is different based on the attack type. 
  - the key value is attacker Offering type (money, service or unknown), if the subregion is reconnaissance.
  - the key value is Attacker Relationship (friend of friend, stranger, or unknown), if the subregion is Social Engineering.
  - the key value is URL Label (good, bad, or NaN), if the subregion is Malware.
  - the key value is Urgency of the attack email (0 to 2), if the subregion is Credential Phishing.
  


In [2]:
def GenerateDictList(filename): ## generate dictionary which consist of the key values of "key", "region", "subregion"
    featureTableWithCluster = pd.read_csv(filename,index_col=0)
    length = len(featureTableWithCluster)
    ZoomableTreeMapJson = []
    for i in range(length):
        if featureTableWithCluster.loc[i,"AttackType:Reconnaissance"] == True:
            ReconnDict = {}
            ReconnDict["key"] = "Attacker Offering = " + str(featureTableWithCluster.loc[i,"Attacker Offering"])
            ReconnDict["region"] =  featureTableWithCluster.loc[i,"jaccardCluster"]
            ReconnDict["subregion"] = "Reconnaissance"
            ZoomableTreeMapJson.append(ReconnDict)
        if featureTableWithCluster.loc[i,"AttackType:SocialEngineering"] == True:
            SocEngDict = {}
            SocEngDict["key"] = "Attacker Relationship = " + str(featureTableWithCluster.loc[i,"Attacker Relationship"])
            SocEngDict["region"] =  featureTableWithCluster.loc[i,"jaccardCluster"]
            SocEngDict["subregion"] = "Social Engineering"
            ZoomableTreeMapJson.append(SocEngDict)
        if featureTableWithCluster.loc[i,"AttackType:Malware"] == True:
            MalwareDict = {}
            MalwareDict["key"] = "URL Label = " + str(featureTableWithCluster.loc[i,"URL Label"])
            MalwareDict["region"] =  featureTableWithCluster.loc[i,"jaccardCluster"]
            MalwareDict["subregion"] = "Malware"
            ZoomableTreeMapJson.append(MalwareDict)
        if featureTableWithCluster.loc[i,"AttackType:CredentialPhishing"] == True:
            CrePhiDict = {}
            CrePhiDict["key"] = "Urgency of the attack email = " + str(featureTableWithCluster.loc[i,"Urgency of the attack email"])
            CrePhiDict["region"] =  featureTableWithCluster.loc[i,"jaccardCluster"]
            CrePhiDict["subregion"] = "Credential Phishing"
            ZoomableTreeMapJson.append(CrePhiDict)
    return(ZoomableTreeMapJson)

def sum_counter(LoadFile,SaveAs): ## count the number which has the same "key", "region", and "subregion". add the number in the dictionary as a "value".
    DictList = GenerateDictList(LoadFile)
    ListCopy = copy.deepcopy(DictList)
    for i in ListCopy:
        i['value'] = sum([1 for j in ListCopy if (j['key'] == i['key'] and j['region'] == i['region'] and j['subregion'] == i['subregion'])])
    finalList = []
    for i in range(len(ListCopy)):
        if ListCopy[i] not in ListCopy[i + 1:]:
            finalList.append(ListCopy[i])
    with open(SaveAs,"w") as f:
        json.dump(finalList,f)

In [3]:
sum_counter("featureTable_withCluster.csv", "ZoomableTreeMapJson_SumValues.json")

# D3 visualization
Pie Chart

Pie Chart need two columns from our featureTable from HW1.
One is URL Label and the other is URL Classification.
extract only "good" and "bad" from URL Label and change the empty values of URL Classification as Unspecified.

In [4]:
def generatePieChart(filename,SaveAs): ## load "URL Label" and "URL Classification" columns from our featureTable. extract only "good" and "bad" from URL Label and change the empty values of URL Classification as Unspecified.
    featureTableWithCluster = pd.read_csv(filename,index_col=0)
    PieChartInput = featureTableWithCluster.loc[featureTableWithCluster["URL Label"].isnull() == False,["URL Label","URL Classification"]].fillna("Unclassified")
    PieChartInput.to_csv(SaveAs,index = False)

In [5]:
generatePieChart("featureTable_withCluster.csv", "PieChartInput.csv")

# D3 visualization
Grouped Bar Chart

Grouped Bar Chart need a pivot table consist of misspelling level (or random capitalization level) and the estimated age. 
in the table, index is made of mis spelling level (or random capitalization level) and the column value is made of estimated age. each cell value represents the number of emails of specific mis spelling level of specific age.


In [6]:
def generateGroupedBarChart(filename,SaveAs,ColumnName): # generate pivot table of estimated age and mis spelling level (or random capital level) 
    featureTableWithCluster = pd.read_csv(filename,index_col=0)
    featureTableWithCluster["count"]=1
    FeatureTableColumnName = "AttackerLanguageStyle:" + ColumnName
    for i in range(len(featureTableWithCluster)):
        if featureTableWithCluster.loc[i,FeatureTableColumnName] == 1:
          featureTableWithCluster.loc[i,ColumnName] = "1 (low)"
        elif featureTableWithCluster.loc[i,FeatureTableColumnName] == 2:
          featureTableWithCluster.loc[i,ColumnName] = "2 (medium)"
        elif featureTableWithCluster.loc[i,FeatureTableColumnName] == 3:
          featureTableWithCluster.loc[i,ColumnName] = "3 (high)"
    featureTableWithCluster["Estimated Age"] = featureTableWithCluster["Estimated Age"].astype(int)
    pivotTable = featureTableWithCluster.pivot_table(values="count",index=[ColumnName],columns=["Estimated Age"],aggfunc=np.sum).fillna(0)
    pivotTable = pivotTable.reset_index()
    pivotTable.to_csv(SaveAs,index = False)

In [7]:
generateGroupedBarChart("featureTable_withCluster.csv","misspellingInput.csv","misspelling")
generateGroupedBarChart("featureTable_withCluster.csv","randomCapitalInput.csv","randomCapital")

# D3 visualization
Heatmap

In this step we generate four different input file for each attacker type.  
input file consist of day, hour and the value (the number of emails)


In [8]:
def generateHeatmap(filename,SaveAs,attacktype): # generate input file for heatmap.
    featureTableWithCluster = pd.read_csv(filename,index_col=0)
    length = len(featureTableWithCluster)
    Heatmap = []
    attacktype = "AttackType:" + attacktype
    for i in range(length): # find a day and hour of each email and put it in a dictionary
        try:
            if featureTableWithCluster.loc[i,attacktype] == True:
                AttackDict = {}
                DateAndTime = featureTableWithCluster.loc[i,"DateAndTime:Date"] +" "+featureTableWithCluster.loc[i,"DateAndTime:time"]
                MyDate = datetime.strptime(DateAndTime, "%Y-%m-%d %H:%M")
                AttackDict["day"] = MyDate.weekday() + 1
                AttackDict["hour"] =  MyDate.hour + 1
                Heatmap.append(AttackDict)
        except:
            pass    

    ListCopy = copy.deepcopy(Heatmap)
    for i in ListCopy: ## count for the emails which sent on a same "day", and an "hour"
        i['value'] = sum([1 for j in ListCopy if (j['day'] == i['day'] and j['hour'] == i['hour'])])
    finalList = []
    for i in range(len(ListCopy)): ## remove duplicate
        if ListCopy[i] not in ListCopy[i + 1:]:
            finalList.append(ListCopy[i])
    Heatmap_TSV = sorted(finalList, key = lambda i: (i['day'], i['hour']))
    keys = Heatmap_TSV[0].keys()
    with open(SaveAs, 'w', newline='')  as f:
        dict_writer = csv.DictWriter(f, keys, delimiter='\t')
        dict_writer.writeheader()
        dict_writer.writerows(Heatmap_TSV)

In [9]:
generateHeatmap("featureTable_withCluster.csv","Heatmap_Reconn.tsv","Reconnaissance")
generateHeatmap("featureTable_withCluster.csv","Heatmap_SocEng.tsv","SocialEngineering")
generateHeatmap("featureTable_withCluster.csv","Heatmap_Malware.tsv","Malware")
generateHeatmap("featureTable_withCluster.csv","Heatmap_CrePhi.tsv","CredentialPhishing")


# D3 visualization
word Scatter plot

input for scatter plot consist of word, the number of word from attacker emails, and the number of word from victim emails.




In [32]:
def WordFromSentence(sentence): # remove title and the sender, and split every words from the contents and remove stop words
    sentence = "\n".join(sentence.split("\n")[2:]) # remove title and sender
    StopWords = set(stopwords.words('english')) 
    WordTokens = word_tokenize(sentence) 
    WordsWithoutStopWords = [w.lower() for w in WordTokens if not w in StopWords] 
    
    WordsDedup = []
    [WordsDedup.append(x) for x in WordsWithoutStopWords if x not in WordsDedup]
    return(WordsDedup)

def WordCounts(EmailList): # count the number of each word from email list
    WordCount = {}
    for i in range(len(EmailList)):
        WordList = WordFromSentence(EmailList[i])
        for word in WordList:
            WordCount[word] = WordCount.get(word, 0) + 1
    return(WordCount)

def compareWordCount(attackerWords,victimWords): # generate a list of dictionary. each dictionary has word, the number of that word from attacker, and the number of that word from victim
    finalResult = []
    attackerWordsList = list(attackerWords.keys())
    victimWordsList = list(victimWords.keys())
    totalWordList = attackerWordsList + victimWordsList
    totalWordList = list(dict.fromkeys(totalWordList))
    for i in range(len(totalWordList)):
        word = totalWordList[i]
        attackerCount = attackerWords.get(word, 0)
        victimCount = victimWords.get(word, 0)
        wordDict = {}
        wordDict["word"] = word 
        wordDict["attacker"] = attackerCount
        wordDict["victim"] = victimCount
        wordDict["totalCount"] = attackerCount + victimCount
        finalResult.append(wordDict)
    return(finalResult)

def SeparateAttack(df): # classify the email contents whether it is from the attacker or the victim
    attacker = []
    victim = []
    for i in range(len(df)):
        if df.loc[i,"AttackType_index"][-1] == '1': # attacker from victim
            attacker.append(df.loc[i,"EmailText"])
        elif df.loc[i,"AttackType_index"][-1] == '2':
            victim.append(df.loc[i,"EmailText"])
    return(attacker,victim)

def generateWordScatterPlot(filename,SaveAs): # generate the final input file for scatter plot
    NewAttacks = pd.read_csv(filename,sep="\t",index_col=0)
    attacker, victim = SeparateAttack(NewAttacks)  
    attackerWords = WordCounts(attacker)
    victimWords = WordCounts(victim)
    wordJson = compareWordCount(attackerWords,victimWords)
    wordtable = pd.DataFrame(wordJson, columns=['word', 'attacker', 'victim','totalCount']).sort_values("totalCount", ascending=False,ignore_index=True)

    ### after remove stop words, we manually deleted below words(ignore) from our final table. ###
    ignore=['.', ',', 'i', ':', 'please', '(', ')', 'the', 'from', 'us', '@', 'would', 'to', 'know', 'this', 'dear', 'my', 'if', 'regards', 'we', 'also', 'mr.', '?', 'one', 'your', 'you', "'s", '-', ';', '%', 'may', 'in', 'of', 'it', 'next', 'as', 'for', 'make', 'and', 'give', 'let', 'since', 'good', 'come', 'subject', 'yours', 'on', 'sincerely', 'mail', '--', '&', 'e-mail']
    wordtable = wordtable[~wordtable.word.isin(ignore)]
    wordtable = wordtable.iloc[0:50,0:3]

    wordtable.to_csv(SaveAs,index = False)

In [33]:
generateWordScatterPlot("new_attacks.tsv","wordtable.csv")